### OLD

In [1]:
import yfinance as yf
import requests
import time
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from datetime import datetime

import numpy as np

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from transformers import BertTokenizer
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datetime import datetime

from transformers import (
    BertTokenizer, 
    BertForSequenceClassification, 
    
)
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
import os

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

/Users/jiunyih/venv/p311_dsa4265/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the search parameters for cryptocurrency-related filings
url = "https://www.sec.gov/cgi-bin/browse-edgar"
params = {
    "company": "Grayscale Bitcoin Cash Trust",  # Searching for Grayscale Bitcoin Cash Trust
    "match": "starts-with",
    "filenum": "",
    "State": "",
    "Country": "",
    "SIC": "",
    "myowner": "exclude",
    "action": "getcompany",
    "type": "10-K", # 10-Q for quarter report    
}

# Add a User-Agent header to comply with SEC's guidelines
headers = {
    "User-Agent": "YourAppName/1.0 (your-email@example.com)"
}

# Send the request to the SEC EDGAR API
response = requests.get(url, headers=headers, params=params)

# Parse the HTML response
soup = BeautifulSoup(response.text, "html.parser")

# Extract the links or filings
filing_links = []
for link in soup.find_all("a", href=True):
    # Only add links that end with "-index.htm"
    if link['href'].endswith("-index.htm"):
        filing_links.append(link['href'])

all_filling_links = []
# Print the filtered links
for filing in filing_links:
    all_filling_links.append(f"https://www.sec.gov{filing}")
    print(f"Filing link: https://www.sec.gov{filing}")

# Add a delay between requests to avoid rate limit issues
time.sleep(1)

Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010-index.htm


In [3]:
docs_dic = {}

for link in all_filling_links:
    try:
        # Generate document link
        doc = link.replace("-index", "").replace("htm", "txt")
        print(f"🔗 Generated Document Link: {doc}")

        # Send the GET request
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check for request errors

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the "Period of Report"
        period_of_report = None
        for div in soup.find_all('div', class_='formGrouping'):
            if "Period of Report" in div.text:
                period_of_report = div.find('div', class_='info').text.strip()

        # Format the period_of_report
        formatted_period_of_report = period_of_report.replace('-', '') if period_of_report else "N/A"
        print(f"✅ Formatted Period of Report: {formatted_period_of_report}")

        # Extract necessary parts from the link
        parts = link.split('/')
        cik = parts[6]  
        filing_identifier = parts[7]  

        # Construct the new URL
        new_link = f"https://www.sec.gov/ix?doc=/Archives/edgar/data/{cik}/{filing_identifier}/bchg-{formatted_period_of_report}.htm"
        print(f"🔗 Generated Report Link: {new_link}\n")

        # Store in the dictionary
        docs_dic[period_of_report] = {
            "period_of_report": formatted_period_of_report,
            "report_link": new_link,
            "doc_link": doc
        }

    except Exception as e:
        print(f"⚠️ Error processing {link}: {e}")

# Print the final dictionary
print(docs_dic)

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235.txt
✅ Formatted Period of Report: 20240630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923.txt
✅ Formatted Period of Report: 20230630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017023045923/bchg-20230630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010.txt
✅ Formatted Period of Report: 20220630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017022018010/bchg-20220630.htm

{'2024-06-30': {'period_of_report': '20240630', 'report_link': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm', 'doc_link'

### NEW

In [1]:
from sec_edgar_downloader import Downloader

# Provide your email address (as required by SEC)
dl = Downloader(email_address="your_email@example.com", download_folder="./sec_data", company_name="school")

dl.get(
    "10-K",
    ticker_or_cik = "AAPL",
    after = "2015-01-01"
)

dl.get(
    "10-K",
    ticker_or_cik = "MSFT",
    after = "2015-01-01"
)


10

In [2]:
import requests
import os
import time

CIK = "0000320193"  # Apple
EMAIL = "your_email@example.com"  # Use your real email
SAVE_FOLDER = "apple_10k_txts"
os.makedirs(SAVE_FOLDER, exist_ok=True)

HEADERS = {
    "User-Agent": f"MyApp/1.0 ({EMAIL})"
}

def get_10k_filing_info(cik):
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    res = requests.get(url, headers=HEADERS)
    data = res.json()
    
    info = []
    for i, form in enumerate(data["filings"]["recent"]["form"]):
        if form == "10-K":
            acc_num = data["filings"]["recent"]["accessionNumber"][i]  # e.g., 0000320193-23-000106
            acc_num_clean = acc_num.replace("-", "")  # e.g., 000032019323000106
            info.append((acc_num_clean, acc_num))  # (folder, file)
    return info

def download_txt_file(folder_acc, file_acc):
    url = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{folder_acc}/{file_acc}.txt"
    res = requests.get(url, headers=HEADERS)
    if res.status_code == 200:
        file_path = os.path.join(SAVE_FOLDER, f"{file_acc}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(res.text)
        print(f"Downloaded: {file_path}")
    else:
        print(f"Failed to download: {file_acc} (status code: {res.status_code})")

# === MAIN ===
filings = get_10k_filing_info(CIK)

for folder_acc, file_acc in filings:
    try:
        download_txt_file(folder_acc, file_acc)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error downloading {file_acc}: {e}")


Downloaded: apple_10k_txts/0000320193-24-000123.txt
Downloaded: apple_10k_txts/0000320193-23-000106.txt
Downloaded: apple_10k_txts/0000320193-22-000108.txt
Downloaded: apple_10k_txts/0000320193-21-000105.txt
Downloaded: apple_10k_txts/0000320193-20-000096.txt
Downloaded: apple_10k_txts/0000320193-19-000119.txt
Downloaded: apple_10k_txts/0000320193-18-000145.txt
Downloaded: apple_10k_txts/0000320193-17-000070.txt
Downloaded: apple_10k_txts/0001628280-16-020309.txt
Downloaded: apple_10k_txts/0001193125-15-356351.txt
Downloaded: apple_10k_txts/0001193125-14-383437.txt


In [3]:
import os

# Define directories
txt_folder = 'apple_10k_txts'
html_folder = 'apple_10k_html'

# Create output folder if it doesn't exist
os.makedirs(html_folder, exist_ok=True)

# Process all .txt files
for filename in os.listdir(txt_folder):
    if filename.endswith('.txt'):
        txt_path = os.path.join(txt_folder, filename)
        html_filename = filename.replace('.txt', '.html')
        html_path = os.path.join(html_folder, html_filename)

        # Read .txt (HTML content)
        with open(txt_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Write as .html
        with open(html_path, 'w', encoding='utf-8') as html_file:
            html_file.write(html_content)

        print(f"Converted: {filename} → {html_filename}")

print("All conversions complete.")


Converted: 0000320193-19-000119.txt → 0000320193-19-000119.html
Converted: 0001628280-16-020309.txt → 0001628280-16-020309.html
Converted: 0000320193-17-000070.txt → 0000320193-17-000070.html
Converted: 0001193125-14-383437.txt → 0001193125-14-383437.html
Converted: 0000320193-24-000123.txt → 0000320193-24-000123.html
Converted: 0000320193-21-000105.txt → 0000320193-21-000105.html
Converted: 0000320193-22-000108.txt → 0000320193-22-000108.html
Converted: 0000320193-23-000106.txt → 0000320193-23-000106.html
Converted: 0001193125-15-356351.txt → 0001193125-15-356351.html
Converted: 0000320193-20-000096.txt → 0000320193-20-000096.html
Converted: 0000320193-18-000145.txt → 0000320193-18-000145.html
All conversions complete.


In [ ]:
import re
from bs4 import BeautifulSoup

# Step 1: Load and clean up the HTML
with open('apple_10k_html/0000320193-17-000070.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

# Step 2: Convert to plain text
text = soup.get_text(separator='\n')

# Step 3: Skip everything until the real content begins (after TOC)
# We'll skip everything until the second "Item " occurrence
item_matches = list(re.finditer(r"(Item\s+[0-9A-Za-z]+(?:[A-Z])?)\.", text, flags=re.IGNORECASE))
# item_pattern = r"(Item\s+[0-9A-Za-z]+(?:[A-Z])?)\."


# Use only the real content (skip the TOC)
content_start = item_matches[1].start() if len(item_matches) > 1 else 0
main_text = text[content_start:]

# Step 4: Extract items and their contents
pattern = re.compile(r"(Item\s+[0-9A-Za-z]+(?:[A-Z])?)\.", re.IGNORECASE)
matches = list(pattern.finditer(main_text))


import unicodedata

def normalize_text(s):
    return unicodedata.normalize("NFKC", s).replace('\xa0', ' ').strip()


extracted_clean = {}

for i in range(len(matches)):
    item_title = normalize_text(matches[i].group(1))
    if item_title.lower() == "item 16":
        continue  # Skip Item 16
    start = matches[i].end()
    end = matches[i + 1].start() if i + 1 < len(matches) else len(main_text)
    content = main_text[start:end].strip()
    
    # Clean content: normalize spaces and remove excessive line breaks
    cleaned_content = re.sub(r'\s+', ' ', content)
    
    extracted_clean[item_title] = cleaned_content

# Optional: Preview a cleaned item
for item, content in list(extracted_clean.items())[:3]:
    print(f"\n{item}")
    print("=" * len(item))
    print(content[:2000]) 

extracted_clean



Item 1A
Risk Factors The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and related notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K. The business, financial condition and operating results of the Company can be affected by a number of factors, whether currently known or unknown, including but not limited to those described below, any one or more of which could, directly or indirectly, cause the Company’s actual financial condition and operating results to vary materially from past, or from anticipated future, financial condition and operating results. Any of these factors, in whole or in part, could materially and adversely a

{'Item 1A': 'Risk Factors The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and related notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K. The business, financial condition and operating results of the Company can be affected by a number of factors, whether currently known or unknown, including but not limited to those described below, any one or more of which could, directly or indirectly, cause the Company’s actual financial condition and operating results to vary materially from past, or from anticipated future, financial condition and operating results. Any of these factors, in whole or in part, could materially and adverse

In [ ]:
# section_reference = {
#     "Item 1": "Describes the company’s core operations, major products/services, markets, supply chain, distribution, and key strategies.",
#     "Item 1A": "Lists key risks that may materially affect the business — regulatory, operational, geopolitical, technological, etc.",
#     "Item 1B": "Details any SEC staff comments on prior filings that remain unresolved. Often marked as 'None'.",
#     "Item 2": "Outlines the company’s physical assets — office buildings, manufacturing facilities, retail stores, warehouses, etc.",
#     "Item 3": "Summarizes significant pending legal actions, including litigation, regulatory matters, and IP disputes.",
#     "Item 4": "Applicable to mining companies; discloses safety violations and citations. Often 'Not applicable' for non-mining companies.",
#     "Item 5": "Covers stock exchange listings, dividends, recent repurchases, and shareholder matters.",
#     "Item 6": "Presents key financial metrics over the past 5 years for historical comparison.",
#     "Item 7": "Management explains financial trends, business performance, liquidity, capital resources, and strategy.",
#     "Item 7A": "Describes risks from market exposures — interest rates, foreign exchange, credit, and commodity prices.",
#     "Item 8": "Contains the audited financial statements — income statement, balance sheet, cash flows, notes.",
#     "Item 9": "Covers any switches in auditors or disputes about accounting practices.",
#     "Item 9A": "Management’s assessment of internal controls over financial reporting and disclosure effectiveness.",
#     "Item 9B": "Miscellaneous disclosures not covered elsewhere.",
#     "Item 10": "Info on leadership team, board composition, and corporate governance policies.",
#     "Item 11": "Breakdown of compensation for top executives, including salary, bonuses, stock, and equity incentives.",
#     "Item 12": "Ownership of voting securities by major shareholders, directors, and management.",
#     "Item 13": "Discloses any insider or related-party transactions that may pose conflicts of interest.",
#     "Item 14": "Details the audit and non-audit fees paid to external accounting firms.",
#     "Item 15": "Lists all exhibits (contracts, bylaws, certifications, etc.) filed with the 10-K.",
#     "Item 16": "A brief summary of the report — optional and rarely used."
# }




In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def chunk_10k_sections_and_prepare_ref_list(parsed_sections: dict, company: str, year: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    all_chunks = []

    for section_title, section_text in parsed_sections.items():
        # Clean title (e.g., "ITEM 1A" => "Item 1A")
        normalized_title = section_title.upper().replace(".", "").replace("ITEM", "Item").strip()
        
        # Split into chunks
        section_docs = splitter.create_documents([section_text])
        
        # Attach metadata to each chunk
        for i, doc in enumerate(section_docs):
            doc.metadata = {
                "company": company,
                "year": year,
                "section": normalized_title,
                "chunk_index": i
            }
            all_chunks.append(doc)

    return all_chunks


In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from transformers import pipeline

# Load summarizer
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY", device=-1)
# knkarthick/MEETING_SUMMARY
# google/pegasus-cnn_dailymail
# sshleifer/distilbart-cnn-12-6
def summarize_text(text, max_len=100, min_len=40):
    text = text[:4000]  # Prevent exceeding token limit
    return summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)[0]['summary_text']

def recursive_summarize(text, max_chunk_chars=3500):
    """Break long text into chunks and summarize, then summarize summaries."""
    import textwrap
    if len(text) <= max_chunk_chars:
        return summarize_text(text)
    
    # Split into smaller chunks
    chunks = textwrap.wrap(text, max_chunk_chars)
    chunk_summaries = [summarize_text(chunk) for chunk in chunks]
    
    # Combine and summarize again
    final_input = " ".join(chunk_summaries)
    return summarize_text(final_input)

def chunk_10k_sections_and_prepare_ref_list(parsed_sections: dict, company: str, year: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    all_chunks = []
    section_summaries = {}

    for section_title, section_text in parsed_sections.items():
        normalized_title = section_title.upper().replace(".", "").replace("ITEM", "Item").strip()
        
        section_docs = splitter.create_documents([section_text])
        chunk_summaries = []

        for i, doc in enumerate(section_docs):
            doc.metadata = {
                "company": company,
                "year": year,
                "section": normalized_title,
                "chunk_index": i
            }

            # Summarize each chunk
            if i % 3 == 1:
                chunk_summary = summarize_text(doc.page_content)
                doc.metadata["chunk_summary"] = chunk_summary
                chunk_summaries.append(chunk_summary)
                print(chunk_summary)

            all_chunks.append(doc)

        # Safely summarize the section via recursive summarization
        combined_summary_text = " ".join(chunk_summaries)
        section_summary = recursive_summarize(combined_summary_text)
        

        if "Maybe you might find this helpful." in section_summary:
            for doc in all_chunks:
                if doc.metadata['section'] == normalized_title and doc.metadata['chunk_index'] == 0:
                    section_summary = doc.page_content[:30]
                    break

        section_summaries[normalized_title] = section_summary

        print(section_summary)
        print("__________________________________________")

    return all_chunks, section_summaries





Device set to use cpu


In [45]:
chunks = chunk_10k_sections_and_prepare_ref_list(extracted_clean, company="Apple", year="2017")

# def update_chunk_titles(chunks):
#     for section_title, chunk_list in chunks.items():
#         for i, chunk in enumerate(chunk_list):
#             # Check if the chunk contains the placeholder text
#             if "Maybe you might find this helpful." in chunk.page_content:
#                 # Replace chunk content with the section title
#                 chunk.page_content = section_title
#     return chunks

# # Assuming `chunks` is the dictionary returned by chunk_10k_sections_and_prepare_ref_list
# updated_chunks = update_chunk_titles(chunks)

The following factors could affect the Company’s business, financial condition and operating results:. Global and regional economic conditions. Uncertainty about global economic conditions poses a risk for the Company.
The Company’s financial results could be affected by a number of factors, among them, delays in product development and/or manufacture, product delays, inability of customers, including channel partners, to obtain credit to finance purchases, and changes in fair value of derivative instruments.
Apple Inc. markets certain mobile communication and media devices based on the iOS mobile operating system and also markets related services, including third-party digital content and applications. The Company has a minority market share in the global smartphone market.
The Company supplies computer parts to the market for personal computers and accessories. This market has been contracting and is dominated by computer makers using competing operating systems. The Company's financ

Your max_length is set to 100, but your input_length is only 5. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)
Your max_length is set to 100, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


The business of Apple Inc. is highly competitive and involves a number of risks. The Company supplies computer parts to the market for personal computers and accessories. It sells its products through cellular network carriers, wholesalers, national and regional retailers and value-added resellers. It has a minority market share in the global smartphone market. The availability and quality of third-party digital content and applications for its iOS devices is dependent on the availability of developers and the right to make them
__________________________________________


Your max_length is set to 100, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


Unresolved Staff Comments None
__________________________________________
Properties The Company’s headq
__________________________________________


...

Executive Compensation The inf
__________________________________________
Principal Accounting Fees and 
__________________________________________
Ernst & Young LLP filed a 10-K with the Securities and Exchange Commission (SEC) on behalf of the Registrant. The documents are as follows: (1) Quarterly Financial Information (Unaudited). (2) Financial Statement Schedules.
Apple filed a 10-K form with the Securities and Exchange Commission (SEC) on 6/10/15. The documents are as follows: Officer’s Certificate of the Registrant, dated as of June 10, 2015.
The documents filed by the Registrant are as follows:. 5/11/17 4.19 Officer’s Certificate of the Registrars. 8-K 4.1 9/12/17 10.1 3/13/15 10.3* Employee Stock Purchase Plan.
On August 1, 2013, the Registrant sent an offer letter to Angela Ahrendts. Restricted Stock Unit Award Agreements outstanding as of August 26, 2014. Form of Performance Award Agreement under 2014 Employee Stock Plan effective as of October 5, 2015. 
Apple filed a 10-K f

Your max_length is set to 100, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


Apple Inc. sells its products worldwide through its retail stores, online stores and direct sales force. It provides Mac and iOS operating systems. iPod touch is a digital music and media player. Apple Pay is a cashless payment service. Apple Inc. filed a 2017 Form 10-K with the SEC.
__________________________________________


Your max_length is set to 100, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


Security Ownership of Certain 
__________________________________________
Certain Relationships and Rela
__________________________________________


In [47]:
chunks[1]

{'Item 1A': 'The business of Apple Inc. is highly competitive and involves a number of risks. The Company supplies computer parts to the market for personal computers and accessories. It sells its products through cellular network carriers, wholesalers, national and regional retailers and value-added resellers. It has a minority market share in the global smartphone market. The availability and quality of third-party digital content and applications for its iOS devices is dependent on the availability of developers and the right to make them',
 'Item 1B': 'Unresolved Staff Comments None',
 'Item 2': 'Properties The Company’s headq',
 'Item 3': 'The Company settled certain matters during the fourth quarter of 2017 that did not have a material impact on the Company’s financial condition or operating results. See the risk factor “ The Company could be impacted by unfavorable results of legal proceedings, such as being found to have infringed on intellectual property rights” in Part I, Ite